## Program requirements:
1. Input is a satellite image in tiff format

2. Image should be sliced for varying zoom levels, pixel density, size

3. It should be sliced according to given number of slices, zoom levels, 

4. We should be able to retrieve the following information from image names: city, zoom level, pixel density, geocoordinate of topleft pixel using the following format: `city_originalImagename_latLongTopLeft_zoomLevel_pixelDensity.jpeg`

## Approach
1. Read tiff image using rasterio
2. Set values for zoom levels, pixel density, image size in pixels
3. Slice image according to the formerly set values
4. Assign geocoordinate of top left pixel for each image slice
5. Rename sliced image and save in prescribed format

## Helper functions required
1. __zoomlevels__ (measured in pixels per meter) to extract zoom level of image. Image slicing preserves zoom levels.
2. __coordRasterTopLeft__ calculates the geocoordinates of top left pixel of image

In [216]:
#import OS for system operations
import os

#importing necessary libraries
from PIL import Image

#disables upper limit for pixel size
Image.MAX_IMAGE_PIXELS = None

#rasterio is mostly used for reading and writing rasters
import rasterio as rio

#earthpy makes it easier to plot and manipulate spatial data in python
import earthpy as et

In [217]:
im = Image.open('image.tif')

In [218]:
im.save('test.jpeg')

In [219]:
with rio.open('image.tif') as sat:
    print(sat.bounds)
    print(sat.meta)

BoundingBox(left=517150.0, bottom=734768.0, right=525342.0, top=742960.0)
{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': 0.0, 'width': 16384, 'height': 16384, 'count': 3, 'crs': CRS.from_dict(init='epsg:32631'), 'transform': Affine(0.5, 0.0, 517150.0,
       0.0, -0.5, 742960.0)}


In [220]:
def zoomlevel(image):
    """Calculates zoom level in meters per pixel"""
    return (image.bounds[3] - image.bounds[1])/image.meta['width']

In [221]:
zoomlevel(sat)

0.5

In [222]:
def coordRasterTopLeft(image_tif):
    """Calculates the coordinate of top left pixel of raster"""
    top_left_lat, top_left_long = image_tif.transform*(0,0)
    return top_left_lat, top_left_long
    print(top_left_lat, top_left_long)

In [223]:
def slicer(infile, chopsize, geoimage):
    """Slices images into square pieces of given chopsize"""
    #infile = 'test.jpeg'
    #chopsize = 300

    img = Image.open(infile)
    width, height = img.size

    # Save Chops of original image
    for x0 in range(0, width, chopsize):
       for y0 in range(0, height, chopsize):
          box = (x0, y0,
                 x0+chopsize if x0+chopsize <  width else  width - 1,
                 y0+chopsize if y0+chopsize < height else height - 1)
          print('%s %s' % (infile, box))
          lat_ = coordRasterTopLeft(geoimage)[0] + x0*zoomlevel(geoimage)
          long_ = coordRasterTopLeft(geoimage)[1] + y0*zoomlevel(geoimage)
          zoomlevels = zoomlevel(geoimage)
          print(zoomlevels)
          
          #long_ = fff
          img.crop(box).save('city_%s_lat%03d_long%03d_zl%03d_pd%d.jpg' % (infile.replace('.jpeg',''), 
                                                                           lat_, 
                                                                           long_, 
                                                                           zoomlevels, chopsize))

In [224]:
slicer('test.jpeg',12000,sat)

test.jpeg (0, 0, 12000, 12000)
0.5
test.jpeg (0, 12000, 12000, 16383)
0.5
test.jpeg (12000, 0, 16383, 12000)
0.5
test.jpeg (12000, 12000, 16383, 16383)
0.5


In [225]:
#shows existing files in working directory
import os

for root, dirs, files in os.walk("."):
    for filename in files:
        print(filename)

city_test_lat517150_long742960_zl000_pd12000.jpg
city_test_lat517150_long748960_zl000_pd12000.jpg
city_test_lat523150_long742960_zl000_pd12000.jpg
city_test_lat523150_long748960_zl000_pd12000.jpg
image.tif
satslice.ipynb
test.jpeg
satslice-checkpoint.ipynb


Zoom level of image slices can be changed by changing zoom level of input image before slicing as slicing retains zoom levels.


In [226]:
#example
resized = im.resize((400,400),Image.ANTIALIAS)


In [227]:
print(resized.size)

(400, 400)
